---
## 1. Teoretický Úvod

### 1.1 Hypotéza Efektivních Trhů (EMH)

Eugene Fama (1970) definoval tři formy tržní efektivity:

| Forma | Informace v ceně | Možnost překonat trh |
|-------|------------------|----------------------|
| **Slabá** | Historické ceny | Technická analýza nefunguje |
| **Středně silná** | + Veřejné informace | Fundamentální analýza nefunguje |
| **Silná** | + Insider informace | Nikdo nemůže překonat trh |

### 1.2 Proč Kombinujeme Technickou a Fundamentální Analýzu?

Empirické studie (Bettman et al., 2009; Kahn & Chun, 2019) ukazují, že:

1. **Technické indikátory** zachycují krátkodobé cenové vzory
2. **Fundamentální metriky** reflektují dlouhodobou hodnotu
3. **Kombinace** obou přístupů zlepšuje predikční schopnost

### 1.3 Architektura Pipeline

```
┌─────────────────────────────────────────────────────────────────┐
│                    DATA COLLECTION PIPELINE                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  Yahoo Finance API                                               │
│       │                                                          │
│       ├──► OHLCV Data (10 let) ──► Technické Indikátory         │
│       │         │                        │                       │
│       │         └────────────────────────┼──► Feature Matrix    │
│       │                                  │         │             │
│       └──► Fundamenty (1.5 roku) ────────┘         │             │
│                                                    ▼             │
│                                          Google Drive Storage    │
└─────────────────────────────────────────────────────────────────┘
```

---
## 2. Setup Prostředí

In [ ]:
# ============================================================
# INSTALACE KNIHOVEN (pro Google Colab)
# ============================================================
!pip install yfinance pandas numpy scikit-learn joblib matplotlib seaborn tqdm -q

print("✓ Knihovny nainstalovány")

In [ ]:
# ============================================================
# IMPORT KNIHOVEN
# ============================================================
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import time
import warnings
from tqdm.notebook import tqdm
import os
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

# Verze knihoven
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")
print(f"yfinance: {yf.__version__}")
print(f"\n✓ Všechny knihovny úspěšně načteny!")

In [ ]:
# ============================================================
# PŘIPOJENÍ GOOGLE DRIVE (pro persistence dat)
# ============================================================
try:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Základní cesta na Drive
    DRIVE_PATH = '/content/drive/MyDrive/MachineLearning'
    os.makedirs(f"{DRIVE_PATH}/data/ohlcv", exist_ok=True)
    os.makedirs(f"{DRIVE_PATH}/data/fundamentals", exist_ok=True)
    os.makedirs(f"{DRIVE_PATH}/data/complete", exist_ok=True)
    os.makedirs(f"{DRIVE_PATH}/models", exist_ok=True)
    
    RUNNING_ON_COLAB = True
    print(f"✓ Google Drive připojen: {DRIVE_PATH}")
except:
    DRIVE_PATH = '.'
    RUNNING_ON_COLAB = False
    os.makedirs("data/ohlcv", exist_ok=True)
    os.makedirs("data/fundamentals", exist_ok=True)
    os.makedirs("data/complete", exist_ok=True)
    os.makedirs("models", exist_ok=True)
    print("ℹ️ Lokální prostředí (ne Colab)")

---
## 3. Konfigurace Projektu

In [ ]:
# ============================================================
# KONFIGURACE
# ============================================================

# Časové období
START_DATE = "2015-01-01"  # 10 let historie
END_DATE = "2025-12-31"

# Výstupní cesty
OUTPUT_BASE = f"{DRIVE_PATH}/data" if RUNNING_ON_COLAB else "data"
OHLCV_DIR = f"{OUTPUT_BASE}/ohlcv"
FUND_DIR = f"{OUTPUT_BASE}/fundamentals"

# Sektory a jejich tickery (50 akcií na sektor = 150 celkem)
SECTORS = {
    "Technology": [
        "AAPL", "MSFT", "GOOGL", "META", "NVDA", "AMD", "INTC", "CRM", "ORCL", "ADBE",
        "CSCO", "AVGO", "TXN", "QCOM", "IBM", "NOW", "INTU", "AMAT", "MU", "LRCX",
        "ADI", "KLAC", "SNPS", "CDNS", "MCHP", "FTNT", "PANW", "CRWD", "ZS", "DDOG",
        "NET", "SNOW", "MDB", "TEAM", "OKTA", "HUBS", "WDAY", "SPLK", "VMW", "DELL",
        "HPQ", "HPE", "NTAP", "WDC", "STX", "KEYS", "ANSS", "CPRT", "PAYC", "CTSH"
    ],
    "Consumer": [
        "AMZN", "TSLA", "HD", "NKE", "MCD", "SBUX", "TGT", "LOW", "TJX", "ROST",
        "COST", "WMT", "DG", "DLTR", "BBY", "ORLY", "AZO", "ULTA", "LULU", "GPS",
        "ANF", "URBN", "RL", "PVH", "TPR", "VFC", "HBI", "GRMN", "POOL", "WSM",
        "RH", "W", "ETSY", "EBAY", "BKNG", "EXPE", "MAR", "HLT", "LVS", "MGM",
        "WYNN", "CCL", "RCL", "NCLH", "DRI", "CMG", "YUM", "QSR", "WING", "SHAK"
    ],
    "Industrials": [
        "CAT", "DE", "BA", "HON", "UPS", "FDX", "LMT", "RTX", "GD", "NOC",
        "GE", "MMM", "EMR", "ETN", "ITW", "PH", "ROK", "AME", "CMI", "PCAR",
        "CARR", "OTIS", "JCI", "TT", "LII", "SNA", "SWK", "IR", "XYL", "IDEX",
        "ROP", "IEX", "GGG", "DOV", "FTV", "NDSN", "AOS", "WTS", "LNT", "AWK",
        "WM", "RSG", "WCN", "FAST", "EXPD", "JBHT", "CHRW", "XPO", "ODFL", "SAIA"
    ]
}

# Fundamentální metriky ke stažení
FUNDAMENTAL_METRICS = [
    # Valuace
    'PE', 'PB', 'PS', 'EV_EBITDA',
    # Profitabilita
    'ROE', 'ROA', 'Profit_Margin', 'Operating_Margin', 'Gross_Margin',
    # Finanční zdraví
    'Debt_to_Equity', 'Current_Ratio', 'Quick_Ratio',
    # Růst
    'Revenue_Growth_YoY', 'Earnings_Growth_YoY'
]

print("📊 KONFIGURACE PROJEKTU")
print("="*50)
print(f"📅 Období: {START_DATE} → {END_DATE}")
print(f"📁 Výstup: {OUTPUT_BASE}")
print(f"\n📈 Sektory:")
for sector, tickers in SECTORS.items():
    print(f"   • {sector}: {len(tickers)} akcií")
print(f"\n   Celkem: {sum(len(t) for t in SECTORS.values())} akcií")
print(f"\n📊 Fundamentální metriky: {len(FUNDAMENTAL_METRICS)}")

---
## 4. Stahování OHLCV Dat

### 4.1 Co jsou OHLCV Data?

**OHLCV** = Open, High, Low, Close, Volume

| Atribut | Popis | Použití |
|---------|-------|--------|
| **Open** | Otevírací cena | Gap analýza |
| **High** | Maximum období | Resistance úrovně |
| **Low** | Minimum období | Support úrovně |
| **Close** | Uzavírací cena | Hlavní referenční cena |
| **Volume** | Objem obchodů | Potvrzení trendu |

### 4.2 Proč Měsíční Frekvence?

- **Denní data**: Příliš zašuměná, mnoho falešných signálů
- **Měsíční data**: Vyhlazený trend, lepší pro dlouhodobou predikci
- **Kompatibilita**: Fundamentální data jsou také čtvrtletní/měsíční

In [ ]:
def download_ohlcv_data(tickers: list, sector: str, start: str, end: str) -> pd.DataFrame:
    """
    Stáhne OHLCV data pro seznam tickerů z Yahoo Finance.
    
    Args:
        tickers: Seznam tickerů (např. ['AAPL', 'MSFT'])
        sector: Název sektoru pro označení dat
        start: Počáteční datum (YYYY-MM-DD)
        end: Koncové datum (YYYY-MM-DD)
    
    Returns:
        pd.DataFrame: OHLCV data s tickerem a sektorem
    """
    all_data = []
    failed_tickers = []
    
    for ticker in tqdm(tickers, desc=f"📥 {sector}"):
        try:
            stock = yf.Ticker(ticker)
            df = stock.history(start=start, end=end, interval="1d")
            
            if df.empty:
                failed_tickers.append(ticker)
                continue
            
            df = df.reset_index()
            df['ticker'] = ticker
            df['sector'] = sector
            df.columns = [c.lower().replace(' ', '_') for c in df.columns]
            
            all_data.append(df)
            time.sleep(0.1)  # Rate limiting
            
        except Exception as e:
            failed_tickers.append(ticker)
    
    if failed_tickers:
        print(f"⚠️ Nepodařilo se stáhnout: {failed_tickers}")
    
    if not all_data:
        return pd.DataFrame()
    
    result = pd.concat(all_data, ignore_index=True)
    print(f"✓ Staženo {len(result):,} záznamů pro {len(all_data)} tickerů")
    
    return result

In [ ]:
# ============================================================
# STAŽENÍ OHLCV DAT PRO VŠECHNY SEKTORY
# ============================================================
all_ohlcv = []

for sector, tickers in SECTORS.items():
    print(f"\n{'='*60}")
    print(f"📥 Stahuji {sector} ({len(tickers)} akcií)")
    print(f"{'='*60}")
    
    sector_data = download_ohlcv_data(tickers, sector, START_DATE, END_DATE)
    
    if not sector_data.empty:
        all_ohlcv.append(sector_data)

# Spojení všech dat
ohlcv_df = pd.concat(all_ohlcv, ignore_index=True)

print(f"\n{'='*60}")
print(f"✅ CELKEM DENNÍCH ZÁZNAMŮ: {len(ohlcv_df):,}")
print(f"{'='*60}")

---
## 5. Agregace na Měsíční Data

In [ ]:
def aggregate_to_monthly(df: pd.DataFrame) -> pd.DataFrame:
    """
    Agreguje denní OHLCV data na měsíční frekvenci.
    
    Agregační pravidla:
    - Open: první hodnota měsíce
    - High: maximum měsíce  
    - Low: minimum měsíce
    - Close: poslední hodnota měsíce
    - Volume: součet za měsíc
    """
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    
    monthly_data = []
    
    for (ticker, sector), group in tqdm(df.groupby(['ticker', 'sector']), 
                                        desc="Agregace na měsíce"):
        group = group.set_index('date')
        
        monthly = group.resample('M').agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'volume': 'sum'
        }).dropna()
        
        monthly = monthly.reset_index()
        monthly['ticker'] = ticker
        monthly['sector'] = sector
        
        monthly_data.append(monthly)
    
    return pd.concat(monthly_data, ignore_index=True)

# Agregace
print("🔄 Agregace na měsíční data...")
monthly_ohlcv = aggregate_to_monthly(ohlcv_df)

print(f"\n✓ Měsíčních záznamů: {len(monthly_ohlcv):,}")
print(f"  Období: {monthly_ohlcv['date'].min().strftime('%Y-%m')} → {monthly_ohlcv['date'].max().strftime('%Y-%m')}")
print(f"  Tickerů: {monthly_ohlcv['ticker'].nunique()}")

---
## 6. Výpočet Technických Indikátorů

### 6.1 Teoretické Pozadí

**Technické indikátory** transformují cenová data do interpretovatelných signálů:

#### RSI (Relative Strength Index)

$$RSI = 100 - \frac{100}{1 + RS}$$

kde $RS = \frac{\text{průměr kladných změn}}{\text{průměr záporných změn}}$

- RSI > 70: **Překoupeno** (sell signal)
- RSI < 30: **Přeprodáno** (buy signal)

#### MACD (Moving Average Convergence Divergence)

$$MACD = EMA_{12} - EMA_{26}$$

$$Signal = EMA_9(MACD)$$

- MACD > Signal: **Bullish** crossover
- MACD < Signal: **Bearish** crossover

#### Moving Averages

**SMA (Simple Moving Average):**
$$SMA_n = \frac{1}{n}\sum_{i=0}^{n-1} P_{t-i}$$

**EMA (Exponential Moving Average):**
$$EMA_t = \alpha \cdot P_t + (1-\alpha) \cdot EMA_{t-1}$$

kde $\alpha = \frac{2}{n+1}$

In [ ]:
def calculate_rsi(prices: pd.Series, period: int = 14) -> pd.Series:
    """
    Výpočet Relative Strength Index (RSI).
    
    RSI = 100 - (100 / (1 + RS))
    RS = průměr kladných změn / průměr záporných změn
    """
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi


def calculate_macd(prices: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
    """
    Výpočet MACD (Moving Average Convergence Divergence).
    
    MACD Line = EMA_fast - EMA_slow
    Signal Line = EMA_signal(MACD Line)
    Histogram = MACD Line - Signal Line
    """
    ema_fast = prices.ewm(span=fast, adjust=False).mean()
    ema_slow = prices.ewm(span=slow, adjust=False).mean()
    
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    histogram = macd_line - signal_line
    
    return macd_line, signal_line, histogram


def add_technical_indicators(df: pd.DataFrame) -> pd.DataFrame:
    """
    Přidá kompletní sadu technických indikátorů.
    
    Přidané features:
    - returns: měsíční výnos
    - volatility_12m: 12-měsíční volatilita
    - rsi_14: RSI s periodou 14
    - macd, macd_signal, macd_hist: MACD indikátory
    - sma_3, sma_6, sma_12: Simple Moving Averages
    - ema_3, ema_6, ema_12: Exponential Moving Averages
    - volume_change: měsíční změna objemu
    - price_momentum: 3-měsíční momentum
    """
    result = []
    
    for ticker, group in tqdm(df.groupby('ticker'), desc="Počítám indikátory"):
        group = group.sort_values('date').copy()
        
        # Výnosy a volatilita
        group['returns'] = group['close'].pct_change()
        group['volatility_12m'] = group['returns'].rolling(window=12).std()
        
        # RSI
        group['rsi_14'] = calculate_rsi(group['close'], period=14)
        
        # MACD
        macd, signal, hist = calculate_macd(group['close'])
        group['macd'] = macd
        group['macd_signal'] = signal
        group['macd_hist'] = hist
        
        # Moving Averages
        for period in [3, 6, 12]:
            group[f'sma_{period}'] = group['close'].rolling(window=period).mean()
            group[f'ema_{period}'] = group['close'].ewm(span=period, adjust=False).mean()
        
        # Volume a momentum
        group['volume_change'] = group['volume'].pct_change()
        group['price_momentum'] = group['close'].pct_change(periods=3)
        
        result.append(group)
    
    return pd.concat(result, ignore_index=True)

# Přidání technických indikátorů
print("📊 Výpočet technických indikátorů...")
monthly_ohlcv = add_technical_indicators(monthly_ohlcv)

tech_cols = [c for c in monthly_ohlcv.columns if c not in ['date', 'ticker', 'sector', 'open', 'high', 'low', 'close', 'volume']]
print(f"\n✓ Přidáno {len(tech_cols)} technických indikátorů:")
for col in tech_cols:
    print(f"   • {col}")

In [ ]:
# Uložení OHLCV dat s technickými indikátory
ohlcv_path = f"{OHLCV_DIR}/all_sectors_ohlcv_10y.csv"
monthly_ohlcv.to_csv(ohlcv_path, index=False)

print(f"💾 OHLCV data uložena: {ohlcv_path}")
print(f"   Záznamů: {len(monthly_ohlcv):,}")
print(f"   Sloupců: {len(monthly_ohlcv.columns)}")

---
## 7. Stahování Fundamentálních Dat

### 7.1 Teoretické Pozadí

**Fundamentální analýza** hodnotí vnitřní hodnotu akcie pomocí finančních metrik:

| Kategorie | Metriky | Co měří |
|-----------|---------|--------|
| **Valuace** | P/E, P/B, P/S, EV/EBITDA | Jak drahá je akcie? |
| **Profitabilita** | ROE, ROA, Marže | Jak efektivně firma vydělává? |
| **Finanční zdraví** | Debt/Equity, Current Ratio | Jak je firma zadlužená? |
| **Růst** | Revenue Growth, Earnings Growth | Jak rychle firma roste? |

### 7.2 Problém Neúplnosti Dat

**Yahoo Finance API poskytuje fundamentální data pouze za posledních ~1.5 roku.**

To vytváří problém pro 10-letou analýzu → řešíme v Notebooku 02 pomocí **Random Forest imputace**.

In [ ]:
def download_fundamentals(tickers: list, sector: str) -> pd.DataFrame:
    """
    Stáhne fundamentální metriky pro seznam tickerů.
    
    Stahované metriky (14 celkem):
    - Valuace: P/E, P/B, P/S, EV/EBITDA
    - Profitabilita: ROE, ROA, marže (3)
    - Finanční zdraví: Debt/Equity, Current Ratio, Quick Ratio
    - Růst: Revenue Growth YoY, Earnings Growth YoY
    """
    all_data = []
    failed_tickers = []
    
    for ticker in tqdm(tickers, desc=f"📊 {sector} fundamenty"):
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            record = {
                'date': datetime.now().strftime('%Y-%m-%d'),
                'ticker': ticker,
                'sector': sector,
                
                # Valuace
                'PE': info.get('trailingPE', info.get('forwardPE', np.nan)),
                'PB': info.get('priceToBook', np.nan),
                'PS': info.get('priceToSalesTrailing12Months', np.nan),
                'EV_EBITDA': info.get('enterpriseToEbitda', np.nan),
                
                # Profitabilita
                'ROE': info.get('returnOnEquity', np.nan),
                'ROA': info.get('returnOnAssets', np.nan),
                'Profit_Margin': info.get('profitMargins', np.nan),
                'Operating_Margin': info.get('operatingMargins', np.nan),
                'Gross_Margin': info.get('grossMargins', np.nan),
                
                # Finanční zdraví
                'Debt_to_Equity': info.get('debtToEquity', np.nan),
                'Current_Ratio': info.get('currentRatio', np.nan),
                'Quick_Ratio': info.get('quickRatio', np.nan),
                
                # Růst
                'Revenue_Growth_YoY': info.get('revenueGrowth', np.nan),
                'Earnings_Growth_YoY': info.get('earningsGrowth', np.nan)
            }
            
            all_data.append(record)
            time.sleep(0.2)  # Rate limiting
            
        except Exception as e:
            failed_tickers.append(ticker)
    
    if failed_tickers:
        print(f"⚠️ Nepodařilo se: {failed_tickers}")
    
    return pd.DataFrame(all_data)

In [ ]:
# ============================================================
# STAŽENÍ FUNDAMENTÁLNÍCH DAT
# ============================================================
all_fundamentals = []

for sector, tickers in SECTORS.items():
    print(f"\n{'='*60}")
    print(f"📊 Fundamenty pro {sector}")
    print(f"{'='*60}")
    
    fund_data = download_fundamentals(tickers, sector)
    all_fundamentals.append(fund_data)

# Spojení
fundamentals_df = pd.concat(all_fundamentals, ignore_index=True)

# Uložení
fund_path = f"{FUND_DIR}/all_sectors_fundamentals.csv"
fundamentals_df.to_csv(fund_path, index=False)

print(f"\n{'='*60}")
print(f"✅ FUNDAMENTY STAŽENY")
print(f"{'='*60}")
print(f"💾 Uloženo: {fund_path}")
print(f"   Záznamů: {len(fundamentals_df)}")

---
## 8. Analýza Chybějících Hodnot

In [ ]:
# Analýza chybějících hodnot ve fundamentech
print("⚠️ CHYBĚJÍCÍ HODNOTY VE FUNDAMENTECH")
print("="*50)

missing = fundamentals_df.isnull().sum()
missing_pct = (missing / len(fundamentals_df) * 100).round(1)

missing_df = pd.DataFrame({
    'Metrika': missing.index,
    'Chybí': missing.values,
    'Procent': missing_pct.values
})
missing_df = missing_df[~missing_df['Metrika'].isin(['date', 'ticker', 'sector'])]
missing_df = missing_df.sort_values('Procent', ascending=False)

print(missing_df.to_string(index=False))
print(f"\n💡 Tyto chybějící hodnoty budou imputovány v Notebooku 02")

In [ ]:
# Vizualizace
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Počet záznamů po sektorech
ax1 = axes[0, 0]
sector_counts = monthly_ohlcv.groupby('sector').size()
colors = {'Technology': '#3498db', 'Consumer': '#2ecc71', 'Industrials': '#e74c3c'}
bars = ax1.bar(sector_counts.index, sector_counts.values, 
               color=[colors[s] for s in sector_counts.index])
ax1.set_title('Počet OHLCV Záznamů po Sektorech', fontsize=12, fontweight='bold')
ax1.set_ylabel('Počet záznamů')
for bar, val in zip(bars, sector_counts.values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 100, 
             f'{val:,}', ha='center', fontsize=10)

# 2. Distribuce P/E
ax2 = axes[0, 1]
pe_data = fundamentals_df['PE'].dropna()
pe_data = pe_data[(pe_data > 0) & (pe_data < 100)]
ax2.hist(pe_data, bins=30, color='steelblue', edgecolor='black', alpha=0.7)
ax2.axvline(pe_data.median(), color='red', linestyle='--', linewidth=2,
            label=f'Medián: {pe_data.median():.1f}')
ax2.set_title('Distribuce P/E Ratio', fontsize=12, fontweight='bold')
ax2.set_xlabel('P/E Ratio')
ax2.legend()

# 3. Průměrná cena v čase
ax3 = axes[1, 0]
for sector in SECTORS.keys():
    sector_data = monthly_ohlcv[monthly_ohlcv['sector'] == sector]
    avg_price = sector_data.groupby('date')['close'].mean()
    ax3.plot(avg_price.index, avg_price.values, label=sector, 
             color=colors[sector], linewidth=2)
ax3.set_title('Průměrná Uzavírací Cena po Sektorech', fontsize=12, fontweight='bold')
ax3.set_xlabel('Datum')
ax3.set_ylabel('Průměrná cena ($)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Chybějící hodnoty
ax4 = axes[1, 1]
missing_plot = missing_df[missing_df['Procent'] > 0].head(10)
bars = ax4.barh(missing_plot['Metrika'], missing_plot['Procent'], color='coral')
ax4.set_title('Chybějící Hodnoty ve Fundamentech (%)', fontsize=12, fontweight='bold')
ax4.set_xlabel('Procent chybějících')
ax4.invert_yaxis()

plt.tight_layout()
plt.savefig(f"{OUTPUT_BASE}/data_overview.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\n💾 Graf uložen: {OUTPUT_BASE}/data_overview.png")

---
## 9. Shrnutí a Další Kroky

### ✅ Dokončeno v tomto notebooku:

| Úkol | Status |
|------|--------|
| Stažení OHLCV dat (10 let) | ✅ |
| Agregace na měsíční frekvenci | ✅ |
| Výpočet technických indikátorů | ✅ |
| Stažení fundamentálních dat | ✅ |
| Uložení na Google Drive | ✅ |

### 📁 Vytvořené soubory:

| Soubor | Popis |
|--------|-------|
| `data/ohlcv/all_sectors_ohlcv_10y.csv` | OHLCV + technické indikátory |
| `data/fundamentals/all_sectors_fundamentals.csv` | Fundamentální metriky |
| `data/data_overview.png` | Vizualizace dat |

### ➡️ Další notebook:

**Notebook 02: Trénování Fundamental Predictor**
- Trénování Random Forest pro imputaci chybějících fundamentů
- Vyhodnocení MAE, RMSE, R² pro každou metriku

In [ ]:
# Finální shrnutí
print("="*70)
print("📊 NOTEBOOK 01 - SHRNUTÍ")
print("="*70)

print(f"\n📈 OHLCV Data:")
print(f"   • Záznamů: {len(monthly_ohlcv):,}")
print(f"   • Tickerů: {monthly_ohlcv['ticker'].nunique()}")
print(f"   • Období: {monthly_ohlcv['date'].min().strftime('%Y-%m')} → {monthly_ohlcv['date'].max().strftime('%Y-%m')}")
print(f"   • Features: {len(tech_cols)} technických indikátorů")

print(f"\n📊 Fundamentální Data:")
print(f"   • Záznamů: {len(fundamentals_df)}")
print(f"   • Metrik: {len(FUNDAMENTAL_METRICS)}")
print(f"   • Průměr chybějících: {missing_pct.mean():.1f}%")

print(f"\n✅ Data připravena pro Notebook 02!")